In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [33]:
# object_data = pd.read_csv('./2024_Jul_ob_count_v3.csv')

In [2]:
# Create array of category names as they appear in the detections data. See paper for details of each category.
categories = ['car', 'person', 'trotro', 'stall', 'truck', 'stove', 'motorcycle', 'vendor', 'lorry', 'umbrella', 'bus', 'trash', 'taxi', 'van', 'debris', 'loudspeaker', 'bowl', 'food', 'animal', 'bicycle']

# Column names in the data frame for the number of counts of each category type in an image.
count_cols = [cat+'_counts' for cat in categories]

super_count_cols = ['people'+'_counts', 'small_vehicles'+'_counts', 'two_wheelers'+'_counts', 'large_vehicles'+'_counts', 'refuse'+'_counts', 'market'+'_counts']

all_count_cols = count_cols + super_count_cols

vehicle_categories = ['car', 'trotro', 'truck', 'motorcycle', 'lorry', 'bus', 'taxi', 'van', 'bicycle']

# Define super categories
super_categories = {
    'people': ['person', 'vendor'],
    'small_vehicles': ['car', 'taxi', 'truck'],
    'two_wheelers': ['bicycle', 'motorcycle'],
    'large_vehicles': ['trotro', 'van', 'lorry', 'bus'],
    'refuse': ['trash', 'debris'],
    'market': ['umbrella', 'stall', 'bowl', 'food']
}

In [35]:
# Ensure datetime is in datetime format
object_data['datetime'] = pd.to_datetime(object_data['datetime_rectified'], format='%Y-%m-%d %H:%M:%S')

# Create additional time-related columns
object_data['hour'] = object_data['datetime'].dt.hour
object_data['day'] = object_data['datetime'].dt.dayofweek + 1  # +1 to match R's 1-indexing
object_data['week'] = object_data['datetime'].dt.isocalendar().week
object_data['year'] = object_data['datetime'].dt.year

# Split 'site_id_cam_angle' into 'site_id' and 'camera' columns
object_data[['site_id', 'camera']] = object_data['site_id_cam_angle'].str.split('_', expand=True)

# Fill missing values in 'camera' with 'single'
object_data['camera'].fillna('single', inplace=True)

# Filter data between specified dates
start_date = pd.Timestamp('2019-04-01')
end_date = pd.Timestamp('2024-04-01')
fixed_object_data = object_data[(object_data['datetime'] >= start_date) & (object_data['datetime'] <= end_date)]
fixed_object_data = fixed_object_data[fixed_object_data['view'] == 'clear']
# Display the first few rows of the new dataframe to verify
fixed_object_data.head()


,directory_name_rectified,site_id_cam_angle,view,image_name,datetime_rectified,date_rectified,animal_counts,bicycle_counts,bowl_counts,bus_counts,...,vendor_counts,directory_name_original,datetime_original,datetime,hour,day,week,year,site_id,camera
0,AD_01_03_2024_C22_S15,AD_right,clear,MFDC3070.JPG,2024-03-01 08:32:02,2024-03-01,0,0,0,0,...,0,AD_01_03_2024_C22_S15,2024-03-01 08:32:02,2024-03-01 08:32:02,8,5,9,2024,AD,right
1,AD_01_03_2024_C22_S15,AD_right,clear,MFDC3071.JPG,2024-03-01 08:37:02,2024-03-01,0,1,0,0,...,0,AD_01_03_2024_C22_S15,2024-03-01 08:37:02,2024-03-01 08:37:02,8,5,9,2024,AD,right
2,AD_01_03_2024_C22_S15,AD_right,clear,MFDC3072.JPG,2024-03-01 08:42:02,2024-03-01,0,0,0,0,...,0,AD_01_03_2024_C22_S15,2024-03-01 08:42:02,2024-03-01 08:42:02,8,5,9,2024,AD,right
3,AD_01_03_2024_C22_S15,AD_right,clear,MFDC3073.JPG,2024-03-01 08:47:02,2024-03-01,0,0,0,0,...,0,AD_01_03_2024_C22_S15,2024-03-01 08:47:02,2024-03-01 08:47:02,8,5,9,2024,AD,right
4,AD_01_03_2024_C22_S15,AD_right,clear,MFDC3074.JPG,2024-03-01 08:52:02,2024-03-01,0,0,0,0,...,0,AD_01_03_2024_C22_S15,2024-03-01 08:52:02,2024-03-01 08:52:02,8,5,9,2024,AD,right


In [36]:
# Sum counts for each super category
for super_cat, categories in super_categories.items():
    # Create a column for each supercategory by summing its categories
    fixed_object_data[super_cat + '_counts'] = fixed_object_data[[cat + '_counts' for cat in categories]].sum(axis=1)

In [37]:
# Step 7: Round the 'datetime' to the nearest hour
fixed_object_data['datetime_hour'] = fixed_object_data['datetime'].dt.round('H')

# Step 8: Sum the counts within each hour for each camera at each site
hourly_counts = fixed_object_data.groupby(['site_id', 'camera', 'datetime_hour'])[all_count_cols].sum().reset_index()

# Step 9: Group by the rounded 'datetime', 'date', and 'site_id', then calculate the mean for each object category
hourly_averages = hourly_counts.groupby(['site_id', 'datetime_hour'])[all_count_cols].mean().reset_index()

# Step 10: Add the date column from the rounded 'datetime_hour'
hourly_averages['date'] = hourly_averages['datetime_hour'].dt.date

# Create additional time-related columns
hourly_averages['hour'] = hourly_averages['datetime_hour'].dt.hour
hourly_averages['day'] = hourly_averages['datetime_hour'].dt.dayofweek + 1  # +1 to match R's 1-indexing
hourly_averages['week'] = hourly_averages['datetime_hour'].dt.isocalendar().week
hourly_averages['year'] = hourly_averages['datetime_hour'].dt.year


# Step 11: Reorder the columns to match the requested format
final_columns = ['datetime_hour', 'date', 'site_id', 'hour', 'day', 'week', 'year'] + all_count_cols
hourly_averages = hourly_averages[final_columns]

# Step 12: Rename 'datetime_hour' to 'datetime' to match the final requested column name
hourly_averages.rename(columns={'datetime_hour': 'datetime'}, inplace=True)

# Display the first few rows of the new dataframe to verify
hourly_averages.head()

,datetime,date,site_id,hour,day,week,year,car_counts,person_counts,trotro_counts,...,bowl_counts,food_counts,animal_counts,bicycle_counts,people_counts,small_vehicles_counts,two_wheelers_counts,large_vehicles_counts,refuse_counts,market_counts
0,2019-04-12 10:00:00,2019-04-12,AD,10,5,15,2019,9.0,32.0,3.0,...,0.0,0.0,0.0,0.0,32.0,15.0,3.0,4.0,0.0,0.0
1,2019-04-12 11:00:00,2019-04-12,AD,11,5,15,2019,228.0,219.0,30.5,...,0.5,0.0,0.0,0.5,219.5,290.0,16.5,36.0,0.0,23.0
2,2019-04-12 12:00:00,2019-04-12,AD,12,5,15,2019,314.0,230.5,49.5,...,0.5,0.0,0.0,2.5,232.0,400.0,17.5,58.0,1.0,26.5
3,2019-04-12 13:00:00,2019-04-12,AD,13,5,15,2019,371.5,192.5,43.5,...,1.0,0.0,0.0,0.5,195.0,482.0,13.0,58.5,2.0,24.5
4,2019-04-12 14:00:00,2019-04-12,AD,14,5,15,2019,351.5,205.0,60.5,...,0.5,0.0,0.0,1.0,206.5,432.0,21.0,67.0,0.0,20.0


In [13]:
from tqdm import tqdm
import pandas as pd
import numpy as np

# Create a function to determine the directory pair or single directory for each row
def get_directory_name(row):
    # Filter rows for the same site and datetime
    matching_rows = fixed_object_data[(fixed_object_data['site_id'] == row['site_id']) & 
                                      (fixed_object_data['datetime_hour'] == row['datetime'])]
    # Get unique directories and cameras
    unique_dirs = matching_rows['directory_name_rectified'].unique()
    unique_cameras = matching_rows['camera'].unique()
    
    if len(unique_dirs) == 1:
        return unique_dirs[0]  # Single camera or only one camera available
    else:
        return '|'.join(sorted(unique_dirs))  # Join directories with '|' to indicate pairs

# Initialize the tqdm progress bar
tqdm.pandas()

# Apply the function to create the new column with progress monitoring
hourly_averages['directory_pair'] = hourly_averages.progress_apply(get_directory_name, axis=1)

# Create indicator variables for left and right cameras
hourly_averages['left_cam'] = hourly_averages.progress_apply(
    lambda row: 1 if 'left' in fixed_object_data[(fixed_object_data['site_id'] == row['site_id']) & 
                                                 (fixed_object_data['datetime_hour'] == row['datetime'])]['camera'].unique() else 0,
    axis=1
)

hourly_averages['right_cam'] = hourly_averages.progress_apply(
    lambda row: 1 if 'right' in fixed_object_data[(fixed_object_data['site_id'] == row['site_id']) & 
                                                  (fixed_object_data['datetime_hour'] == row['datetime'])]['camera'].unique() else 0,
    axis=1
)


  0%|▏                                                                                                                                                             | 445/353738 [09:23<124:11:21,  1.27s/it]


KeyboardInterrupt: 

In [38]:
# Display the first few rows of the new dataframe to verify
hourly_averages_cleaned = hourly_averages.loc[:,~hourly_averages.columns.duplicated()].copy()
print(hourly_averages.columns)
print("\n")
print(hourly_averages_cleaned.columns)

Index(['datetime', 'date', 'site_id', 'hour', 'day', 'week', 'year',
       'car_counts', 'person_counts', 'trotro_counts', 'stall_counts',
       'truck_counts', 'stove_counts', 'motorcycle_counts', 'vendor_counts',
       'lorry_counts', 'umbrella_counts', 'bus_counts', 'trash_counts',
       'taxi_counts', 'van_counts', 'debris_counts', 'loudspeaker_counts',
       'bowl_counts', 'food_counts', 'animal_counts', 'bicycle_counts',
       'people_counts', 'small_vehicles_counts', 'two_wheelers_counts',
       'large_vehicles_counts', 'refuse_counts', 'market_counts'],
      dtype='object')


Index(['datetime', 'date', 'site_id', 'hour', 'day', 'week', 'year',
       'car_counts', 'person_counts', 'trotro_counts', 'stall_counts',
       'truck_counts', 'stove_counts', 'motorcycle_counts', 'vendor_counts',
       'lorry_counts', 'umbrella_counts', 'bus_counts', 'trash_counts',
       'taxi_counts', 'van_counts', 'debris_counts', 'loudspeaker_counts',
       'bowl_counts', 'food_counts', 

In [11]:
# hourly_averages.to_csv('hourly_averages_indicator.csv')

In [3]:
# hourly_averages.to_csv('hourly_averages_updated.csv')

hourly_averages = pd.read_csv('hourly_averages_v2.csv')

In [5]:
hourly_averages.head()

,Unnamed: 0,datetime,date,site_id,hour,day,week,year,car_counts,person_counts,...,animal_counts,bicycle_counts,people_counts,small_vehicles_counts,two_wheelers_counts,large_vehicles_counts,refuse_counts,market_counts,directory_pair,fold
0,1,2019-04-12 11:00:00,2019-04-12,AD,11,5,15,2019,228.0,219.0,...,0.0,0.5,219.5,290.0,16.5,36.0,0.0,23.0,AD_12_04_2019_C14|AD_12_04_2019_C17,4
1,2,2019-04-12 12:00:00,2019-04-12,AD,12,5,15,2019,314.0,230.5,...,0.0,2.5,232.0,400.0,17.5,58.0,1.0,26.5,AD_12_04_2019_C14|AD_12_04_2019_C17,5
2,3,2019-04-12 13:00:00,2019-04-12,AD,13,5,15,2019,371.5,192.5,...,0.0,0.5,195.0,482.0,13.0,58.5,2.0,24.5,AD_12_04_2019_C14|AD_12_04_2019_C17,3
3,4,2019-04-12 14:00:00,2019-04-12,AD,14,5,15,2019,351.5,205.0,...,0.0,1.0,206.5,432.0,21.0,67.0,0.0,20.0,AD_12_04_2019_C14|AD_12_04_2019_C17,5
4,5,2019-04-12 15:00:00,2019-04-12,AD,15,5,15,2019,354.5,249.5,...,0.0,0.5,251.0,433.0,23.5,54.0,0.5,24.0,AD_12_04_2019_C14|AD_12_04_2019_C17,5


In [4]:
# Get the value counts of the 'directory_pair' column
directory_pair_counts = hourly_averages['directory_pair'].value_counts()

# Filter for directory_pair values that occur only once
single_occurrence_pairs = directory_pair_counts[directory_pair_counts == 1]

# Get the number of such unique directory_pair values
count_single_occurrences = single_occurrence_pairs.shape[0]

print(f"Number of 'directory_pair' values that occur only once: {count_single_occurrences}")


Number of 'directory_pair' values that occur only once: 0


In [55]:
# Step 1: Get the value counts of the 'directory_pair' column
directory_pair_counts = hourly_averages['directory_pair'].value_counts()

# Step 2: Identify the 'directory_pair' values that occur only once
single_occurrence_pairs = directory_pair_counts[directory_pair_counts == 1].index

# Step 3: Filter the dataframe to exclude these 'directory_pair' values
hourly_averages_filtered = hourly_averages[~hourly_averages['directory_pair'].isin(single_occurrence_pairs)]

# If you want to update the original DataFrame, you can assign it back
hourly_averages = hourly_averages_filtered

# Display the resulting DataFrame
hourly_averages_filtered.head()

,Unnamed: 0,datetime,date,site_id,hour,day,week,year,car_counts,person_counts,...,food_counts,animal_counts,bicycle_counts,people_counts,small_vehicles_counts,two_wheelers_counts,large_vehicles_counts,refuse_counts,market_counts,directory_pair
1,1,2019-04-12 11:00:00,2019-04-12,AD,11,5,15,2019,228.0,219.0,...,0.0,0.0,0.5,219.5,290.0,16.5,36.0,0.0,23.0,AD_12_04_2019_C14|AD_12_04_2019_C17
2,2,2019-04-12 12:00:00,2019-04-12,AD,12,5,15,2019,314.0,230.5,...,0.0,0.0,2.5,232.0,400.0,17.5,58.0,1.0,26.5,AD_12_04_2019_C14|AD_12_04_2019_C17
3,3,2019-04-12 13:00:00,2019-04-12,AD,13,5,15,2019,371.5,192.5,...,0.0,0.0,0.5,195.0,482.0,13.0,58.5,2.0,24.5,AD_12_04_2019_C14|AD_12_04_2019_C17
4,4,2019-04-12 14:00:00,2019-04-12,AD,14,5,15,2019,351.5,205.0,...,0.0,0.0,1.0,206.5,432.0,21.0,67.0,0.0,20.0,AD_12_04_2019_C14|AD_12_04_2019_C17
5,5,2019-04-12 15:00:00,2019-04-12,AD,15,5,15,2019,354.5,249.5,...,0.5,0.0,0.5,251.0,433.0,23.5,54.0,0.5,24.0,AD_12_04_2019_C14|AD_12_04_2019_C17


In [56]:
len(hourly_averages_filtered)

354828

In [57]:
hourly_averages_filtered.to_csv('hourly_averages_v2.csv')

In [58]:
hourly_averages_filtered['directory_pair'].value_counts()

directory_pair
LA_27_03_2020_C16_S28                            597
EL_29_03_2020_C12_S30                            524
AD_24_04_2020_C12_S17|AD_24_04_2020_C4_S21       505
JT_24_04_2020_C31_S49                            505
NIMA_24_04_2020_C27_S45|NIMA_24_04_2020_C2_S2    504
                                                ... 
EL_15_06_2020_C12_S53|EL_15_06_2020_C32_S65        2
NIMA_15_11_2019_C2_S28                             2
JT_18_12_2020_C9_S5                                2
JT_03_01_2020_C31_S44                              2
TMW_06_09_2021_C19_S5|TMW_06_09_2021_C30_S44       2
Name: count, Length: 2612, dtype: int64

In [59]:
import pandas as pd
import numpy as np

# Assuming you have already read your data into a dataframe
# hourly_averages_filtered is the dataframe

# Randomly assign each row to one of five folds
np.random.seed(42)  # For reproducibility
hourly_averages['fold'] = np.random.randint(1, 6, size=len(hourly_averages_filtered))

year                2019  2020  2021  2022  2023  2024
fold hour day week                                    
1    0    1   1        2     0     3     0     3     3
              2        0     0     2     3     4     2
              3        0     3     2     5     1     1
              4        0     1     1     2     2     0
              5        0     0     0     1     1     0
...                  ...   ...   ...   ...   ...   ...
5    23   7   49       1     1     1     5     1     0
              50       1     0     5     0     2     0
              51       1     2     1     1     1     0
              52       2     2     0     1     4     0
              53       0     0     2     0     0     0

[44388 rows x 6 columns]


In [16]:
for fold in range(1,6):
    for col in ['hour', 'day', 'week', 'year']:
        print(hourly_averages[hourly_averages['fold'] == fold][[col]].value_counts())

hour
14      3064
10      3040
3       2983
18      2979
6       2974
20      2973
15      2972
16      2968
12      2953
22      2952
17      2951
9       2950
21      2938
23      2934
13      2928
11      2928
1       2927
2       2923
19      2910
8       2908
4       2908
0       2906
7       2901
5       2884
Name: count, dtype: int64
day
2      10291
6      10211
3      10153
1      10057
5      10046
7      10023
4       9973
Name: count, dtype: int64
week
34      1640
28      1592
26      1538
31      1529
32      1520
30      1494
29      1488
37      1484
33      1481
24      1478
42      1470
35      1467
43      1461
41      1461
39      1456
40      1454
36      1448
47      1435
27      1428
49      1422
45      1418
38      1413
50      1408
21      1406
51      1405
22      1404
44      1395
46      1394
19      1382
23      1382
18      1381
20      1377
2       1373
25      1365
52      1352
17      1341
1       1300
48      1296
16      1287
8       1281
7       123

In [34]:
import pandas as pd

# Assuming 'hourly_averages' is your DataFrame with the 'site_id', 'fold', and 'directory_pair', 'hour', 'day', 'week', 'year' columns

# Step 1: Group by 'site_id', 'fold', and 'directory_pair' and count the occurrences
site_fold_directory_pair_counts = hourly_averages.groupby(['site_id', 'fold', 'directory_pair']).size().reset_index(name='count')

# Step 2: Identify the 'site_id', 'fold', and 'directory_pair' combinations that occur only 5 times or less
single_occurrence_pairs = site_fold_directory_pair_counts[site_fold_directory_pair_counts['count'] <= 5][['site_id', 'fold', 'directory_pair']]

# Intermediate Step: Print the number of single occurrences before removal for each site
for site in single_occurrence_pairs['site_id'].unique():
    num_single_occurrences = single_occurrence_pairs[single_occurrence_pairs['site_id'] == site].shape[0]
    print(f"Number of 'directory_pair' values that occur 5 or fewer times within each fold for site {site}: {num_single_occurrences}")

# Initialize a list to collect pairs to actually remove
pairs_to_remove = []

# Check for each site and fold if removing the pair would still ensure that the full range of hour, day, week, and year exists
for index, row in tqdm(single_occurrence_pairs.iterrows()):
    site = row['site_id']
    fold = row['fold']
    directory_pair = row['directory_pair']
    
    # Subset the data to the specific site and fold
    subset_data = hourly_averages[(hourly_averages['site_id'] == site) & (hourly_averages['fold'] == fold)]
    
    # Get the unique values for hour, day, week, and year before removal
    unique_values_before = {
        'hour': set(subset_data['hour'].unique()),
        'day': set(subset_data['day'].unique()),
        'week': set(subset_data['week'].unique()),
        'year': set(subset_data['year'].unique())
    }
    
    # Subset data to exclude the current directory_pair
    subset_data_after_removal = subset_data[subset_data['directory_pair'] != directory_pair]
    
    # Get the unique values for hour, day, week, and year after removal
    unique_values_after = {
        'hour': set(subset_data_after_removal['hour'].unique()),
        'day': set(subset_data_after_removal['day'].unique()),
        'week': set(subset_data_after_removal['week'].unique()),
        'year': set(subset_data_after_removal['year'].unique())
    }
    
    # Check if any unique value is lost after removal
    if all(unique_values_after[key] == unique_values_before[key] for key in unique_values_before):
        pairs_to_remove.append((site, fold, directory_pair))
    else:
        # Print a warning if removing would eliminate a unique value
        lost_values = {key: unique_values_before[key] - unique_values_after[key] for key in unique_values_before if unique_values_after[key] != unique_values_before[key]}
        print(f"WARNING: Removing 'directory_pair' = {directory_pair} from site {site}, fold {fold} would remove the following unique values: {lost_values}")

# Step 3: Convert pairs_to_remove into a DataFrame
pairs_to_remove_df = pd.DataFrame(pairs_to_remove, columns=['site_id', 'fold', 'directory_pair'])

# Intermediate Step: Print the number of pairs that will be removed after the check
print(f"Number of 'directory_pair' values that will actually be removed: {len(pairs_to_remove_df)}")

# Step 4: Merge the original DataFrame with the pairs_to_remove_df to identify rows to remove
hourly_averages_filtered = pd.merge(hourly_averages, pairs_to_remove_df, on=['site_id', 'fold', 'directory_pair'], how='left', indicator=True)

# Step 5: Filter out the rows where '_merge' column is 'both' (indicating pairs to remove)
hourly_averages_filtered = hourly_averages_filtered[hourly_averages_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])

# If you want to update the original DataFrame, you can assign it back
hourly_averages = hourly_averages_filtered

# Display the resulting DataFrame
hourly_averages_filtered.head()

# Save the filtered DataFrame to CSV (if needed)
hourly_averages_filtered.to_csv('hourly_averages_v3.csv', index=False)


Number of 'directory_pair' values that occur 5 or fewer times within each fold for site AD: 114
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site ASH: 64
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site EL: 139
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site JT: 55
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site LA: 22
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site N1W: 120
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site NM: 87
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site TF: 148
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site TMW: 100
Number of 'directory_pair' values that occur 5 or fewer times within each fold for site UGH: 30


879it [00:20, 42.53it/s]


Number of 'directory_pair' values that will actually be removed: 879
